In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn


# MLflow config
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # eller din serveradresse
mlflow.set_experiment("creditcard_fraud_detection")

# Load data
data_path = r"C:\Job_og_eksamensbevis\Github\projekter\RF_project\data\creditcard.csv"
df = pd.read_csv(data_path)

X = df.drop('Class', axis=1)
y = df['Class']

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

precisions = []
recalls = []
f1s = []

# === CV TRÆNING + MLflow logging ===
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_val, y_train_val), 1):
    X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
    y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    rf = RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train_resampled, y_train_resampled)
    y_val_pred = rf.predict(X_val)

    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)

    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

    # MLflow logging for fold
    with mlflow.start_run(run_name=f"Fold_{fold}"):
        mlflow.log_param("n_estimators", 100)
        mlflow.log_param("sampler", "SMOTE")
        mlflow.log_param("fold", fold)

        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        mlflow.sklearn.log_model(rf, f"rf_model_fold_{fold}")

        print(f"\nFold {fold}:")
        print("Confusion Matrix:")
        print(confusion_matrix(y_val, y_val_pred))
        print(f"Precision: {precision:.4f}")
        print(f"Recall:    {recall:.4f}")
        print(f"F1-score:  {f1:.4f}")
        print("Classification report:")
        print(classification_report(y_val, y_val_pred))

print("\n=== Gennemsnitlig performance over 5 folds ===")
print(f"Gns. Precision: {sum(precisions)/len(precisions):.4f}")
print(f"Gns. Recall:    {sum(recalls)/len(recalls):.4f}")
print(f"Gns. F1-score:  {sum(f1s)/len(f1s):.4f}")



2025/07/14 18:50:36 INFO mlflow.tracking.fluent: Experiment with name 'creditcard_fraud_detection' does not exist. Creating a new experiment.
c:\Users\jeppe\anaconda3\a3\envs\ml-env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
2025/07/14 18:51:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/14 18:51:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\jeppe\anaconda3\a3\envs\ml-env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This f


Fold 1:
Confusion Matrix:
[[45481    10]
 [   11    67]]
Precision: 0.8701
Recall:    0.8590
F1-score:  0.8645
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     45491
           1       0.87      0.86      0.86        78

    accuracy                           1.00     45569
   macro avg       0.93      0.93      0.93     45569
weighted avg       1.00      1.00      1.00     45569

🏃 View run Fold_1 at: http://127.0.0.1:5000/#/experiments/2/runs/8e933adeefb54cfd99ce148f6da225df
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


2025/07/14 18:52:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/14 18:52:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\jeppe\anaconda3\a3\envs\ml-env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Fold 2:
Confusion Matrix:
[[45485     5]
 [   18    61]]
Precision: 0.9242
Recall:    0.7722
F1-score:  0.8414
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     45490
           1       0.92      0.77      0.84        79

    accuracy                           1.00     45569
   macro avg       0.96      0.89      0.92     45569
weighted avg       1.00      1.00      1.00     45569

🏃 View run Fold_2 at: http://127.0.0.1:5000/#/experiments/2/runs/7a02ad8be0484ea4a08f651ca2bfae1a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


2025/07/14 18:53:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/14 18:53:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\jeppe\anaconda3\a3\envs\ml-env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


...


Fold 4:
Confusion Matrix:
[[45479    11]
 [   11    68]]
Precision: 0.8608
Recall:    0.8608
F1-score:  0.8608
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     45490
           1       0.86      0.86      0.86        79

    accuracy                           1.00     45569
   macro avg       0.93      0.93      0.93     45569
weighted avg       1.00      1.00      1.00     45569

🏃 View run Fold_4 at: http://127.0.0.1:5000/#/experiments/2/runs/fb69f2d0f0e042ee8a2bf7a18ebb7cd2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


2025/07/14 18:55:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/14 18:56:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\jeppe\anaconda3\a3\envs\ml-env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Fold 5:
Confusion Matrix:
[[45483     7]
 [   15    64]]
Precision: 0.9014
Recall:    0.8101
F1-score:  0.8533
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     45490
           1       0.90      0.81      0.85        79

    accuracy                           1.00     45569
   macro avg       0.95      0.90      0.93     45569
weighted avg       1.00      1.00      1.00     45569

🏃 View run Fold_5 at: http://127.0.0.1:5000/#/experiments/2/runs/0b5a0f1ff55f473c98ddb8da7a32acab
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2

=== Gennemsnitlig performance over 5 folds ===
Gns. Precision: 0.8878
Gns. Recall:    0.8123
Gns. F1-score:  0.8473

=== Endelig evaluering på test-sæt ===
Confusion Matrix:
[[56847    17]
 [   17    81]]
Precision: 0.8265
Recall:    0.8265
F1-score:  0.8265
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.0

2025/07/14 18:57:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/14 18:57:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'creditcard_rf_model'.
2025/07/14 18:57:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: creditcard_rf_model, version 1
Created version '1' of model 'creditcard_rf_model'.


🏃 View run Final_model_training at: http://127.0.0.1:5000/#/experiments/2/runs/992de3d0d8dc4f7d96cdf50a6150f6ab
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [4]:
# === Final model  ===

smote = SMOTE(random_state=42)
X_train_val_resampled, y_train_val_resampled = smote.fit_resample(X_train_val, y_train_val)

final_rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
final_rf.fit(X_train_val_resampled, y_train_val_resampled)

y_test_pred = final_rf.predict(X_test)

precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print("\n=== Endelig evaluering på test-sæt ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")
print("Classification report:")
print(classification_report(y_test, y_test_pred))


with mlflow.start_run(run_name="Final_model_training"):
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("sampler", "SMOTE")
    mlflow.log_param("model_type", "RandomForest")

    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Her logger vi modellen KUN som artifact – uden at registrere den i MLflow Model Registry
    mlflow.sklearn.log_model(
        sk_model=final_rf,
        artifact_path="model"
    )

c:\Users\jeppe\anaconda3\a3\envs\ml-env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



=== Endelig evaluering på test-sæt ===
Confusion Matrix:
[[56847    17]
 [   17    81]]
Precision: 0.8265
Recall:    0.8265
F1-score:  0.8265
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.83      0.83        98

    accuracy                           1.00     56962
   macro avg       0.91      0.91      0.91     56962
weighted avg       1.00      1.00      1.00     56962



2025/07/14 19:19:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/14 19:19:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Final_model_training at: http://127.0.0.1:5000/#/experiments/2/runs/208c8b6e5eb74a8980f20727e5883a58
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [ ]:
# tilføj navn til den model jeg vil registrere (tilføj challenger eller champion i alias)
model_name = "RF-SMOTE"
run_id = input("Indtast run-id: ")
model_uri = f"runs:/{run_id}/"{model_name}
result = mlflow.register_model(
    model_uri, model_name
)